<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read" data-toc-modified-id="Read-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read</a></span></li><li><span><a href="#Test-data" data-toc-modified-id="Test-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test data</a></span></li><li><span><a href="#basic" data-toc-modified-id="basic-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>basic</a></span></li><li><span><a href="#mission-dim" data-toc-modified-id="mission-dim-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>mission dim</a></span></li><li><span><a href="#Option-To-xarray-3Dim" data-toc-modified-id="Option-To-xarray-3Dim-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Option To xarray 3Dim</a></span></li><li><span><a href="#Time-dim*" data-toc-modified-id="Time-dim*-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Time dim*</a></span></li></ul></div>

In [3]:
%pylab inline
import os
import sys
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob 

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# Read

In [4]:
path_home = "../data/extract_database_Guyane_Bresil.txt"
df = pd.read_csv(path_home, sep="|", skipinitialspace = True)
df = df.rename(columns=lambda x: x.strip())
df["num_station"] = df.num_station.str.strip()
df["mission"] = df.mission.str.strip()
df["heure"] = df.heure.str.strip()
df["jour"] = df.jour.str.strip()
df = df.replace(-999, np.nan)
df = df.loc[~df["num_station"].isnull(), ]  #type: ignore
df.loc[(df.heure.isnull())==True,"heure"] = "00:00:00"
df["time"]  = df.jour + "T" + df.heure
df["time"] = pd.to_datetime(df["time"],format='%Y-%m-%dT%H:%M:%S')
df = df[df.duplicated() == False] # 26
df = df.loc[((df.chla.isnull() == False) | (df.mes.isnull() == False) | (df.doc.isnull() == False)| (df.poc.isnull() == False)),]
df2 = df.copy()
df["coord"] = df[['lat', 'lon', 'time', 'profondeur']].apply(tuple, axis=1)
#df = df.set_index("time")

# Test data 

In [5]:
df

,mission,num_station,jour,heure,lat,lon,profondeur,chla,mes,doc,poc,time,coord
1,Guyane2009,MODIS-A,2009-10-15,17:05:00,5.774000,-53.344000,1.0,4.949642,3.90,NaN,NaN,2009-10-15 17:05:00,"(5.774, -53.344, 2009-10-15 17:05:00, 1.0)"
2,Guyane2009,T2S1,2009-10-13,14:37:00,4.678000,-51.940000,1.0,11.249554,325.56,NaN,NaN,2009-10-13 14:37:00,"(4.678, -51.94, 2009-10-13 14:37:00, 1.0)"
3,Guyane2009,T2S3,2009-10-13,16:30:00,4.743000,-51.963000,1.0,5.849822,10.47,NaN,NaN,2009-10-13 16:30:00,"(4.743, -51.963, 2009-10-13 16:30:00, 1.0)"
4,Guyane2009,T2S4,2009-10-13,13:11:00,4.815400,-51.944000,1.0,2.605422,10.80,NaN,NaN,2009-10-13 13:11:00,"(4.8154, -51.944, 2009-10-13 13:11:00, 1.0)"
5,Guyane2009,T2S5,2009-10-13,12:20:00,4.851000,-51.917000,1.0,1.837993,10.67,NaN,NaN,2009-10-13 12:20:00,"(4.851, -51.917, 2009-10-13 12:20:00, 1.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,Trans_Atlantico,TA29,2009-11-17,07:32:00,-30.000834,-48.960667,1.0,0.146336,NaN,NaN,NaN,2009-11-17 07:32:00,"(-30.000834, -48.960667, 2009-11-17 07:32:00, ..."
3217,Trans_Atlantico,TA30,2009-11-17,08:45:00,-30.005333,-49.150166,1.0,0.169864,NaN,NaN,NaN,2009-11-17 08:45:00,"(-30.005333, -49.150166, 2009-11-17 08:45:00, ..."
3218,Trans_Atlantico,TA31,2009-11-17,10:00:00,-30.006666,-49.343334,1.0,0.129234,NaN,NaN,NaN,2009-11-17 10:00:00,"(-30.006666, -49.343334, 2009-11-17 10:00:00, ..."
3219,Trans_Atlantico,TA32,2009-11-17,12:08:00,-30.002832,-49.643665,1.0,0.127874,NaN,NaN,NaN,2009-11-17 12:08:00,"(-30.002832, -49.643665, 2009-11-17 12:08:00, ..."


In [6]:
df.loc[(df.lat == 7.73) & (df.lon == -47.8),] #type: ignore

,mission,num_station,jour,heure,lat,lon,profondeur,chla,mes,doc,poc,time,coord
91,guyaplac,32,2003-05-28,00:00:00,7.73,-47.8,0.0,0.00008,NaN,NaN,NaN,2003-05-28,"(7.73, -47.8, 2003-05-28 00:00:00, 0.0)"
92,guyaplac,33,2003-05-28,00:00:00,7.73,-47.8,0.0,0.00013,NaN,NaN,NaN,2003-05-28,"(7.73, -47.8, 2003-05-28 00:00:00, 0.0)"


In [7]:
df.loc[(df.lat == -3.628397 ) & (df.lon == -31.980950),] #type: ignore

,mission,num_station,jour,heure,lat,lon,profondeur,chla,mes,doc,poc,time,coord
933,Abracos_1,A1-3,2015-10-01,17:24:00,-3.628397,-31.98095,2.9834,49.1,NaN,NaN,NaN,2015-10-01 17:24:00,"(-3.628397, -31.98095, 2015-10-01 17:24:00, 2...."
934,Abracos_1,A1-3,2015-10-01,17:24:00,-3.628397,-31.98095,2.9834,44.1,NaN,NaN,NaN,2015-10-01 17:24:00,"(-3.628397, -31.98095, 2015-10-01 17:24:00, 2...."
935,Abracos_1,A1-3,2015-10-01,17:24:00,-3.628397,-31.98095,2.9834,5.0,NaN,NaN,NaN,2015-10-01 17:24:00,"(-3.628397, -31.98095, 2015-10-01 17:24:00, 2...."


In [8]:
df.loc[(df.lat == -22.868450 ) & (df.lon == -43.212580 ),] #type: ignore

,mission,num_station,jour,heure,lat,lon,profondeur,chla,mes,doc,poc,time,coord
861,Guanabara,G1005,2014-04-17,10:40:00,-22.86845,-43.21258,1.0,35.31075,25.0,292.5,NaN,2014-04-17 10:40:00,"(-22.86845, -43.21258, 2014-04-17 10:40:00, 1.0)"
2225,Guanabara,G1005,2014-04-17,10:40:00,-22.86845,-43.21258,1.0,75.15000,NaN,NaN,NaN,2014-04-17 10:40:00,"(-22.86845, -43.21258, 2014-04-17 10:40:00, 1.0)"


In [9]:
df.loc[df.coord.duplicated()==True,] #type: ignore

,mission,num_station,jour,heure,lat,lon,profondeur,chla,mes,doc,poc,time,coord
92,guyaplac,33,2003-05-28,00:00:00,7.730000,-47.800000,0.0000,0.00013,NaN,NaN,NaN,2003-05-28 00:00:00,"(7.73, -47.8, 2003-05-28 00:00:00, 0.0)"
934,Abracos_1,A1-3,2015-10-01,17:24:00,-3.628397,-31.980950,2.9834,44.10000,NaN,NaN,NaN,2015-10-01 17:24:00,"(-3.628397, -31.98095, 2015-10-01 17:24:00, 2...."
935,Abracos_1,A1-3,2015-10-01,17:24:00,-3.628397,-31.980950,2.9834,5.00000,NaN,NaN,NaN,2015-10-01 17:24:00,"(-3.628397, -31.98095, 2015-10-01 17:24:00, 2...."
937,Abracos_1,A1-4,2015-10-02,09:36:00,-3.901568,-32.341038,2.9834,52.10000,NaN,NaN,NaN,2015-10-02 09:36:00,"(-3.901568, -32.341038, 2015-10-02 09:36:00, 2..."
938,Abracos_1,A1-4,2015-10-02,09:36:00,-3.901568,-32.341038,2.9834,0.00000,NaN,NaN,NaN,2015-10-02 09:36:00,"(-3.901568, -32.341038, 2015-10-02 09:36:00, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648,Guanabara,G426,2013-08-22,11:53:00,-22.943733,-43.139830,1.0000,28.38500,NaN,NaN,NaN,2013-08-22 11:53:00,"(-22.943733, -43.13983, 2013-08-22 11:53:00, 1.0)"
1928,Guanabara,G708,2013-12-10,10:46:00,-22.807817,-43.142033,1.0000,26.51500,NaN,NaN,NaN,2013-12-10 10:46:00,"(-22.807817, -43.142033, 2013-12-10 10:46:00, ..."
2040,Guanabara,G820,2014-01-24,07:34:00,-22.729616,-43.067867,1.0000,182.56500,NaN,NaN,NaN,2014-01-24 07:34:00,"(-22.729616, -43.067867, 2014-01-24 07:34:00, ..."
2131,Guanabara,G911,2014-01-20,02:27:00,-22.734383,-43.074135,1.0000,2.09350,NaN,NaN,NaN,2014-01-20 02:27:00,"(-22.734383, -43.074135, 2014-01-20 02:27:00, ..."


# basic

In [10]:
df2 = df2.loc[:,[ "chla", "lat", "lon", "doc", "poc", "mes", "profondeur", "time"]]
df2 = df2.set_index("time")
df2.to_xarray()

<xarray.Dataset>
Dimensions:     (time: 3034)
Coordinates:
  * time        (time) datetime64[ns] 2009-10-15T17:05:00 ... 2009-11-17T13:1...
Data variables:
    chla        (time) float64 4.95 11.25 5.85 2.605 ... 0.1292 0.1279 0.152
    lat         (time) float64 5.774 4.678 4.743 4.815 ... -30.01 -30.0 -30.0
    lon         (time) float64 -53.34 -51.94 -51.96 ... -49.34 -49.64 -49.81
    doc         (time) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    poc         (time) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    mes         (time) float64 3.9 325.6 10.47 10.8 10.67 ... nan nan nan nan
    profondeur  (time) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

# mission dim

In [11]:
df2.set_index(['mission', 'time'], inplace=True)
df2

KeyError: "None of ['mission', 'time'] are in the columns"

In [ ]:
df2 = df2.loc[:,[ "chla", "lat", "lon", "doc", "poc", "mes", "profondeur"]]

In [ ]:
df2

In [ ]:
df3 = df2.loc[~df2.index.duplicated()]
xdf = df3.to_xarray()

In [ ]:
xdf

In [ ]:
xdf.chla[8,:]

In [ ]:
xdf.chla.sel(mission='Guyane2009')

# Option To xarray 3Dim

In [ ]:
df2["coord"] = df2[['lat', 'lon', 'time']].apply(tuple, axis=1)
df2 = df2.loc[df2.coord.duplicated()==False,]
df2 = df2.reset_index()
df2 = df2.loc[:,["lat", "lon", "time", "chla"]]

In [ ]:
df2.set_index(['lat', 'lon', 'time'], inplace=True)

In [ ]:
df2

NameError: name 'df2' is not defined

In [12]:
df2.to_xarray()

<xarray.Dataset>
Dimensions:     (time: 3034)
Coordinates:
  * time        (time) datetime64[ns] 2009-10-15T17:05:00 ... 2009-11-17T13:1...
Data variables:
    chla        (time) float64 4.95 11.25 5.85 2.605 ... 0.1292 0.1279 0.152
    lat         (time) float64 5.774 4.678 4.743 4.815 ... -30.01 -30.0 -30.0
    lon         (time) float64 -53.34 -51.94 -51.96 ... -49.34 -49.64 -49.81
    doc         (time) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    poc         (time) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    mes         (time) float64 3.9 325.6 10.47 10.8 10.67 ... nan nan nan nan
    profondeur  (time) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0

# Time dim*

In [ ]:
df.set_index("time", inplace=True)
df.sort_index(inplace = True)

In [ ]:

# define data with variable attributes
data_vars = {'Chla':(['time'], df.chla.values, 
                         {'units': 'mg/m3', 
                          'long_name':'Chlorophyll-a'}),
            'MES':(['time'], df.mes.values, 
                         {'units': '...', 
                          'long_name':'...'}),
            'POC':(['time'], df.poc.values, 
                         {'units': '...', 
                          'long_name':'...'}),
            'DOC':(['time'], df.doc.values, 
                         {'units': '...', 
                          'long_name':'...'}),
            'latitude':(['time'], df.lat.values, 
                         {'units': '...', 
                          'long_name':'...'}),
            'longitude':(['time'], df.lon.values, 
                         {'units': '...', 
                          'long_name':'...'}),
             'mission':(['time'], df.mission.values, 
                         {'units': '...', 
                          'long_name':'...'}),
             'num_station':(['time'], df.num_station.values, 
                         {'units': '...', 
                          'long_name':'...'}),
             'prof':(['time'], df.profondeur.values, 
                         {'units': '...', 
                          'long_name':'...'})}

# define coordinates
coords= {'time': (['time'], df.index.values)}


# define global attributes
attrs = {'creation_date':"05/06/2023", 
         'author':'Xavier Meriaux', 
         'email':'xavier.meriaux@univ-littoral.fr'}

# create dataset
ds = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)

In [ ]:
ds

In [ ]:
#ds.chla.attrs["coucou"] = "Je suis là"

In [ ]:
ds.Chla

In [ ]:
ds.Chla.sel(time=slice("2005-01-01T00:00:00", "2006-01-01")).plot(ls=":")